In [1]:
from pgvector.psycopg import register_vector
import psycopg
import streamlit as st
from sentence_transformers import SentenceTransformer
import ollama
from langchain.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

Importation des modules nécessaires.

In [2]:
embedding_model = SentenceTransformer("thenlper/gte-large")

Initialisation du modèle d'embedding.

In [3]:
conn = psycopg.connect(
    user="postgres",
    password="postgres",
    host="127.0.0.1",
    port="5432",
    dbname="postgres",
    autocommit=True
)

Connexion à la base de données.

In [4]:
def database_search(query):
    vector = create_vector(query)
    embedding_query = "[" + ",".join(map(str, vector)) + "]"
    
    query_sql = f"""
    SELECT chunk, embedding <#> '{embedding_query}' AS distance
    FROM building_rules_vectors_items
    ORDER BY embedding <#> '{embedding_query}'
    LIMIT 3;
    """
    
    data = conn.execute(query_sql).fetchall()
    result = [row[0] for row in data]
    return result

Définition de la fonction de recherche dans la base de données.

In [5]:
def create_vector(prompt):
    result = embedding_model.encode(prompt)
    return result

Définition de la fonction de création de vecteurs.

In [6]:
def create_prompt(llm_query, database_results):
    content_start = (
        "Repond uniquement en fraincais a la question suivant en te basant sur le contexte ci-dessous.\n\n" +
        "Contexte:\n"
    )

    content_end = f"\n\nQuestion: {llm_query}\nReponse:"

    content = content_start + "\n\n---\n\n".join(database_results) + content_end
    prompt = [{'role': 'user', 'content': content}]
    return prompt

Définition de la fonction de création de prompt.

In [7]:
def create_completion(prompt, ollama_model, llm_query):
    if ollama_model == "mistral":
        completion = ollama.chat(model=ollama_model, messages=prompt)
        return completion['message']['content']
    elif ollama_model == "mistral-building-rules-model-fr":
        llm = ChatOllama(model=ollama_model)
        prompt_template = ChatPromptTemplate.from_messages([("human", "{question}")])
        chain = prompt_template | llm
        ans = chain.invoke({llm_query})
        return ans.content

Définition de la fonction de création de complétion.

In [8]:
with st.form("prompt_form"):
    result = ""
    prompt = ""
    choice = st.selectbox('Select the model you want ?', ('mistral', 'mistral-building-rules-model-fr'))
    ollama_model = choice
    semantic_query = st.text_area("Database prompt:", None)
    llm_query = st.text_area("LLM prompt:", None)
    submitted = st.form_submit_button("Send")
    
    if submitted:
        vector_results = database_search(semantic_query)
        prompt = create_prompt(llm_query, vector_results)
        print(prompt)
        result = create_completion(prompt, ollama_model, llm_query)
    
    e = st.expander("LLM prompt created:")
    e.write(prompt)
    st.write(result)

Interface utilisateur avec Streamlit.